In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(15, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 15),
            nn.Sigmoid()
        )

        self.mu = nn.Linear(32, 32)
        self.logvar = nn.Linear(32, 32)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        x = self.encoder(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Define the loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.BCELoss(reduction='sum')
    reconstruction_loss = BCE(recon_x, x.view(-1, 15))
    KL_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + KL_divergence

In [17]:
import pickle

with open('real_matrix_list.pickle', 'rb') as f:
    matrix_list = pickle.load(f)

matrix_tensor = torch.stack(matrix_list).to("cpu")
matrix_tensor[0]

tensor([[ 0.0587,  1.4593, -1.3316, -1.9080,  0.8360],
        [ 0.2385, -0.1331, -0.1122, -0.6529,  0.2540],
        [-0.3553, -0.6916,  0.7228,  1.2096, -1.1862]])

In [18]:
# Prepare your dataset (replace this with your actual dataset)
# Example random data
data = matrix_tensor

# Normalize the data
data_min = data.min()
data_max = data.max()
data_normalized = (data - data_min) / (data_max - data_min)

# Initialize the model and optimizer
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for matrix in data_normalized:
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(matrix.view(-1, 15))
        loss = loss_function(recon_batch, matrix, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch: {}, Loss: {:.4f}'.format(epoch, train_loss / len(data_normalized)))



Epoch: 0, Loss: 9.1679
Epoch: 1, Loss: 9.1066
Epoch: 2, Loss: 9.0939
Epoch: 3, Loss: 9.0923
Epoch: 4, Loss: 9.0895
Epoch: 5, Loss: 9.0880
Epoch: 6, Loss: 9.0878
Epoch: 7, Loss: 9.0867
Epoch: 8, Loss: 9.0851
Epoch: 9, Loss: 9.0846
Epoch: 10, Loss: 9.0840
Epoch: 11, Loss: 9.0829
Epoch: 12, Loss: 9.0818
Epoch: 13, Loss: 9.0813
Epoch: 14, Loss: 9.0815
Epoch: 15, Loss: 9.0812
Epoch: 16, Loss: 9.0808
Epoch: 17, Loss: 9.0805
Epoch: 18, Loss: 9.0805
Epoch: 19, Loss: 9.0805
Epoch: 20, Loss: 9.0803
Epoch: 21, Loss: 9.0799
Epoch: 22, Loss: 9.0799
Epoch: 23, Loss: 9.0799
Epoch: 24, Loss: 9.0798
Epoch: 25, Loss: 9.0792
Epoch: 26, Loss: 9.0798
Epoch: 27, Loss: 9.0793
Epoch: 28, Loss: 9.0796
Epoch: 29, Loss: 9.0794
Epoch: 30, Loss: 9.0797
Epoch: 31, Loss: 9.0788
Epoch: 32, Loss: 9.0791
Epoch: 33, Loss: 9.0792
Epoch: 34, Loss: 9.0795
Epoch: 35, Loss: 9.0791
Epoch: 36, Loss: 9.0784
Epoch: 37, Loss: 9.0787
Epoch: 38, Loss: 9.0788
Epoch: 39, Loss: 9.0794
Epoch: 40, Loss: 9.0790
Epoch: 41, Loss: 9.0790
Ep